# Goal: Find drivers of upsets in chess games and create a model to predict upsets 

* Upset is defined as a player with a lower rating winning a game agenst a player with a higher rating
* Model should make a predictions after having 'interviewed' each player as to thier intended opening as white and what defence they intend to use against a given opening as black

### Initial Thoughts

<br>

* Going into this project I am of two minds.

<br>

**First**
* Chess is a skill based game with no random elements (except assigning first move). 
* Because of this the player with the highest level of skill will win any game not determined by variation in player performance. 
* Because of this a given game will be won by the player with the highest level of skill a large majority of the time. * If this is true conditions underwhich variation in performance is the highest should result in the highest likelyhood of an upset.

<br>

**Second**
* It may also be the case that more skilled players are able to maintain consistancy better than less skilled players under conditions that would increased variation in thier performance.
* If this is true, those conditions may make upsets less likely as the variance would have a grater effect on the the less skilled player than on the more skilled player.

<br>

**Conclution**
* Though these two schools of thought may point at differing conclutions, both seem grounded in reason and I am eager to see what the data can tell us

### Initial Hypothisese About Drivers

* There will be few instances of upsets, possibly leading to an imbalanced data set
* As ratings for both players increase, the likelyhood of an upset will decrease 
* As the margin between player ratings increase the likelyhood of an upsets will decrease
* Shorter time incraments will increase the likelyhood of an upset
* Unranked games will have a higher likelyhood of an upset than ranked games
* Games where the higher rated player is moving the white pieces (gaining first move advantage) will have a decreased likelyhood of of an upset
* Some opening/defense stratagies may be more or less prone to upsets
* openings/defences that are more popular or perfered by higher rated players may be more/less prone to upset

# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
import sklearn.preprocessing

import warnings
warnings.filterwarnings("ignore")

import re

import wrangle as w

# Acquire

* Data acquired from Kaggle at https://www.kaggle.com/datasnaek/chess
* It contained 20,058 rows and 9 columns before cleaning

# Prepare

**Data was very clean initially, I performed the following steps to insure that is was ready for exploration:**
* Removed columns that did not contain useful information\* 
* Renamed columns to premote readability\*
* Checked for nulls in the data (there were none)
* Checked that column data types were ppropriate
* Removed white space from values in object columns
* There were no rows lost during preperation
* Added Target column 'upset' indicating weather the lower rated player won the game
* Added additional features to investigate (columns that could be calculated one row at a time)*
* Split data into train, validate and test (approx. 60/20/20), stratifying on 'upset'
* Added additional features to investigate (columns that requiered an aggregate calculation by column)
* aggregat calculations were performed on train data
* resulting calculations were then applied to create columns in train, validate, and test data.

\* See data dictionary for full list

In [2]:
df = w.wrangle_chess_data(reprep = True)
train, validate, test = w.split_my_data(df)
train, validate, test = w.fe_post_split(train, validate, test)
train.columns

AttributeError: 'DataFrame' object has no attribute 'rating'

In [ ]:
train[train.game_rating > train.game_rating.mean].opening_code.value_counts()

In [ ]:
# get percent of total games played
train['opening_code_pop'] = train['opening_code'].apply(lambda value : train.opening_code.value_counts()[value]/len(train))
train['opening_name_pop'] = train['opening_name'].apply(lambda value : train.opening_name.value_counts()[value]/len(train))

# get percent of total games played by high level players
train['opening_code_pop'] = train['opening_code'].apply(lambda value : train[train.game_rating > train.game_rating.mean].opening_code.value_counts()[value]/len(train))
train['opening_name_pop'] = train['opening_name'].apply(lambda value : train.opening_name.value_counts()[value]/len(train))

In [ ]:
train.head(10)

In [ ]:
train.head()

In [ ]:
opening_list = train.opening_code.to_list()
opening_name_list = train.opening_name.to_list()

In [ ]:
average_moves_per_game = train.turns.mean()
int(average_moves_per_game)

In [ ]:
median_moves_per_game = train.turns.median()
median_moves_per_game

In [ ]:
train.time_code.value_counts().head(20)

In [ ]:
reg = r'\+'

string = '10+5'

time = re.sub(reg, ' ', string)
time = time.split(' ')

((int(time[0]) * 60) + (int(time[1]) * (average_moves_per_game/2)))/60

In [ ]:
10 * 60

In [ ]:
5 * 30

In [ ]:
750/60

### Examine Object Variables

In [ ]:
# distribution of the data
columns = ['ended_as', 'winning_pieces', 
           'time_code', 'opening_code', 
           'opening_name', 'upset']

for col in columns:
    
    df[col].value_counts().plot(kind='bar', title = f"{col} distribution")
    
    plt.show()

### Takeaways

* Resignations usually happen when mate is enevitable I see no reason to seperate the two
* I wonder if running out of time has an effect on upsets? 
* White does have an advantage, though it is much smaller than I thought it would be, at about 10% higher number of wins than black
* time_code, opening_code, and opening_name, have too many values to sort through at the moment and will have to be binned or pruened
* upsets represent about 1/3 or the data, which is higher than I thought it would be

### Examine Quantitative Variables

In [ ]:
# distribution of the data
cols = ['turns', 'white_rating', 'black_rating']

for col in cols:
    plt.hist(df[col])
    plt.title(col+' distripution')
    plt.show()

### Takeaways

* Turns is slightly right skewed 
* Black and white rating distributions are pretty normally distributed and are nearly if not entierly identical

In [ ]:
df.time_code.value_counts()

### I'm goint to try to prune the object columns by removing the values that do not have a significant represintation I an setting my trial cut off point at 50 or more occurrences 

In [ ]:
df.to_csv('games_preped.csv')
df = pd.read_csv('games_preped.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace = True)

In [ ]:
df.head()

In [ ]:
# distribution of the data
columns = ['ended_as', 'winning_pieces', 
           'time_code', 'opening_code', 
           'opening_name', 'upset']

for col in columns:
    
    df[col].value_counts().plot(kind='bar', title = f"{col} distribution")
    
    plt.show()

In [ ]:
from wrangle import wrangle_chess_data

In [ ]:
df = wrangle_chess_data()
df.head()

In [ ]:
df['rating_diff'] = abs(df.white_rating-df.black_rating)
df.head()

# removing under 1200 rated players to eliminate new/novice players

In [ ]:
df = df[(df.white_rating >= 1200) &(df.black_rating >= 1200)]

In [ ]:
df['average_rating'] = (df.white_rating + df.black_rating) / 2

In [ ]:
df

In [ ]:
df['upset'] = (((df.white_rating > df.black_rating) & (df.winning_pieces == 'black')) |
               ((df.white_rating < df.black_rating) & (df.winning_pieces == 'white')))

In [ ]:
df

In [ ]:
df['underdog_first_move'] = df.white_rating < df.black_rating

In [ ]:
df

In [ ]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.upset)

In [ ]:
train, validate =  train_test_split(train_validate, test_size=.3, random_state=123, stratify=df.upset)

In [ ]:
train.size

In [ ]:
df.to_csv('games_preped.csv')
df = pd.read_csv('games_preped.csv')